<a href="https://colab.research.google.com/github/silverstar0727/NLP_study/blob/main/ch4_LogisticRegression_TFidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd /content/drive/My Drive/kaggle_nlp

/content/drive/My Drive/kaggle_nlp


In [ ]:
import os

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
# TF-IDF를이용한 모델 구현

# scikit learn의 TfidfVectorizer를 사용 (텍스트 형태의 데이터를 인풋)
DATA_IN_PATH = 'after_preprocessing/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

In [ ]:
# min_df: 설정한 값보다 작게 나오면 벡터화 과정에서 제외
# analyzer: 벡터화 단위 여기서는 문자 하나단위 (word는 단어 단위)
# sublinear_tf: 단어의 빈도수에 대한 스무딩(등장 카운트에 더해줌으로써 0이되는 것을 방지)
# ngram_range: 빈도의 기본 단위를 어느 범위의 ngram으로 설정할 지 정하는 것
# max_features: 각 벡터의 최대길이 설정
vectorizer = TfidfVectorizer(min_df = 0.0, analyzer = 'char', sublinear_tf = True, 
                             ngram_range = (1, 3), max_features = 5000)
X = vectorizer.fit_transform(reviews)

In [ ]:
# 학습과 검증 데이터 셋 분리
from sklearn.model_selection import train_test_split
import numpy as np

RANDOM_SEED = 42 # random seed를 고정
TEST_SPLIT = 0.2 # 20퍼를 evaluate data set으로 설정

y = np.array(sentiments) # 정답 배열

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size = TEST_SPLIT,
                                                    random_state = RANDOM_SEED)

In [ ]:
# 모델선언 및 학습(logistic regression)
from sklearn.linear_model import LogisticRegression

# 로지스틱 회귀 객체를 생성
# class_weight: 각 라벨에 대해 균형있게 학습하도록 함
lgs = LogisticRegression(class_weight = 'balanced')
lgs.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
# evaluation data를 이용한 모델 평가
print("Accuracy: %f".format(lgs.score(X_eval, y_eval)))

Accuracy: %f


In [ ]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

In [ ]:
testDataVecs = vectorizer.transform(test_data['review'])

In [ ]:
test_predicted = lgs.predict(testDataVecs)
print(test_predicted)

[1 0 1 ... 0 1 0]


In [ ]:
DATA_OUT_PATH = 'output/'

if not os.path.exists(DATA_OUT_PATH):
  os.makedirs(DATA_OUT_PATH)

ids = list(test_data['id'])
answer_dataset = pd.DataFrame({'id': ids, 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_tfidf_answer.csv')